In [2]:
import numpy as np
import pandas as pd
import jpholiday
import pmdarima as pm
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [3]:
df = pd.read_excel('/Users/hiraokatatsuru/Library/CloudStorage/OneDrive-個人用/ドキュメント/物数データ.xlsx')
df.set_index('date', inplace=True)

In [6]:
# --- 1. データ準備 --------------------------------------------------
# y: 元データの目的変数
y = df['通常物数']

# exogenous に祝日フラグなどを入れる例（省略可）
exog = pd.DataFrame({
    'is_holiday': df.index.to_series().apply(jpholiday.is_holiday).astype(int),
    'dow':        df.index.dayofweek.astype(int)
}, index=df.index)

# --- 2. auto_arima で(p,d,q)を探索 ------------------------------------
stepwise = pm.auto_arima(
    y, exogenous=exog,
    start_p=0, start_q=0,
    max_p=5, max_q=5,
    seasonal=False,  # 季節成分があれば seasonal=True + m=7 など
    trace=True,
    error_action='ignore',
    suppress_warnings=True,
    stepwise=True
)
print(stepwise.summary())

# --- 3. SARIMAX モデルの学習 ------------------------------------------
order = stepwise.order  # (p,d,q)
model = SARIMAX(
    y,
    exog=exog,
    order=order,
    enforce_stationarity=False,
    enforce_invertibility=False
)
res = model.fit(disp=False)
print(res.summary())

# --- 4. バックテスト（ウォークフォワード） ----------------------------
def backtest_arima(y, exog, res, horizon=28):
    history_y = y.copy()
    history_exog = exog.copy()
    preds = []
    for dt in y.index[-horizon:]:
        # その日までで再フィット（本番ではフィットせず update も可）
        m = SARIMAX(history_y, exog=history_exog, order=order).fit(disp=False)
        # exog_future を用意
        exog_dt = exog.loc[[dt]]
        f = m.predict(start=dt, end=dt, exog=exog_dt)[0]
        preds.append(f)
        # 実績 or 予測を追加
        history_y.loc[dt] = f
        history_exog.loc[dt] = exog_dt.loc[dt]
    actual = y.iloc[-horizon:]
    mae  = mean_absolute_error(actual, preds)
    rmse = np.sqrt(mean_squared_error(actual, preds))
    return mae, rmse

mae, rmse = backtest_arima(y, exog, res, horizon=28)
print(f"ARIMA Backtest MAE={mae:.0f}, RMSE={rmse:.0f}")

# --- 5. 将来予測 -------------------------------------------------------
future_dates = pd.date_range(start=df.index[-1] + pd.Timedelta(1, 'd'), periods=28, freq='D')
# 将来 exog の用意
future_exog = pd.DataFrame({
    'is_holiday': [int(jpholiday.is_holiday(d)) for d in future_dates],
    'dow':        [d.weekday() for d in future_dates]
}, index=future_dates).astype(int)

# ダイレクト予測
forecast = res.get_forecast(steps=28, exog=future_exog)
# ① 予測値取得
pred = forecast.predicted_mean

# ② inf を NaN に変換
pred = pred.replace([np.inf, -np.inf], np.nan)

# ③ NaN を埋める（方法は要件に応じて）
#    • 前日や直近の実績で埋めるなら forward‐fill
pred = pred.fillna(method='ffill')
#    • 埋めきれない先頭の NaN はゼロにする例
pred = pred.fillna(0)

# ④ 四捨五入＆整数化
pred_rounded = pred.round(-3).astype(int)

# 結果表示
for d, v in pred_rounded.items():
    wd = ['月','火','水','木','金','土','日'][d.weekday()]
    mark = '祝' if jpholiday.is_holiday(d) else ''
    print(f"{d:%Y-%m-%d} ({wd}{mark}): {v} 通")

/Users/hiraokatatsuru/miniconda3/envs/shift_builder/lib/python3.12/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/hiraokatatsuru/miniconda3/envs/shift_builder/lib/python3.12/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/hiraokatatsuru/miniconda3/envs/shift_builder/lib/python3.12/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/hiraokatatsuru/miniconda3/envs/shift_builder/lib/python3.12/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/hiraokatatsuru/miniconda3/envs/shift_builder/lib/

Performing stepwise search to minimize aic
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=32581.883, Time=0.02 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=32335.643, Time=0.02 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.12 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=32579.885, Time=0.01 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=32011.144, Time=0.02 sec
 ARIMA(3,1,0)(0,0,0)[0] intercept   : AIC=31877.338, Time=0.05 sec
 ARIMA(4,1,0)(0,0,0)[0] intercept   : AIC=31877.083, Time=0.06 sec
 ARIMA(5,1,0)(0,0,0)[0] intercept   : AIC=31729.250, Time=0.08 sec


/Users/hiraokatatsuru/miniconda3/envs/shift_builder/lib/python3.12/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/hiraokatatsuru/miniconda3/envs/shift_builder/lib/python3.12/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/hiraokatatsuru/miniconda3/envs/shift_builder/lib/python3.12/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


 ARIMA(5,1,1)(0,0,0)[0] intercept   : AIC=31225.853, Time=0.20 sec
 ARIMA(4,1,1)(0,0,0)[0] intercept   : AIC=31472.915, Time=0.15 sec


/Users/hiraokatatsuru/miniconda3/envs/shift_builder/lib/python3.12/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/hiraokatatsuru/miniconda3/envs/shift_builder/lib/python3.12/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


...

/Users/hiraokatatsuru/miniconda3/envs/shift_builder/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/hiraokatatsuru/miniconda3/envs/shift_builder/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/hiraokatatsuru/miniconda3/envs/shift_builder/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/hiraokatatsuru/miniconda3/envs/shift_builder/lib/python3.12/site-packages/statsmodels/tsa/statespace/mlemodel.py:3160: UserWarning: Early subset of data for variable 0 has too few non-missing observations to calculate test statistic.
  test_sta

                               SARIMAX Results                                
Dep. Variable:                   通常物数   No. Observations:                 1339
Model:               SARIMAX(5, 1, 2)   Log Likelihood                     nan
Date:                Sat, 14 Jun 2025   AIC                                nan
Time:                        21:40:11   BIC                                nan
Sample:                    10-01-2021   HQIC                               nan
                         - 05-31-2025                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
is_holiday  -6.29e+04        nan        nan        nan         nan         nan
dow        -1.366e+04        nan        nan        nan         nan         nan
ar.L1      -7.864e+12        nan        nan        n

/var/folders/t3/j19g2ldj2ks74jzlzsqq83640000gn/T/ipykernel_30298/249534669.py:46: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  f = m.predict(start=dt, end=dt, exog=exog_dt)[0]
/var/folders/t3/j19g2ldj2ks74jzlzsqq83640000gn/T/ipykernel_30298/249534669.py:49: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-76679.7159310539' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  history_y.loc[dt] = f
/Users/hiraokatatsuru/miniconda3/envs/shift_builder/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/hiraokatatsuru/miniconda3/envs/shift_builder

ARIMA Backtest MAE=15094, RMSE=23105
2025-06-01 (日): 0 通
2025-06-02 (月): 0 通
2025-06-03 (火): 0 通
2025-06-04 (水): 0 通
2025-06-05 (木): 0 通
2025-06-06 (金): 0 通
2025-06-07 (土): 0 通
2025-06-08 (日): 0 通
2025-06-09 (月): 0 通
2025-06-10 (火): 0 通
2025-06-11 (水): 0 通
2025-06-12 (木): 0 通
2025-06-13 (金): 0 通
2025-06-14 (土): 0 通
2025-06-15 (日): 0 通
2025-06-16 (月): 0 通
2025-06-17 (火): 0 通
2025-06-18 (水): 0 通
2025-06-19 (木): 0 通
2025-06-20 (金): 0 通
2025-06-21 (土): 0 通
2025-06-22 (日): 0 通
2025-06-23 (月): 0 通
2025-06-24 (火): 0 通
2025-06-25 (水): 0 通
2025-06-26 (木): 0 通
2025-06-27 (金): 0 通
2025-06-28 (土): 0 通


/var/folders/t3/j19g2ldj2ks74jzlzsqq83640000gn/T/ipykernel_30298/249534669.py:46: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  f = m.predict(start=dt, end=dt, exog=exog_dt)[0]
/var/folders/t3/j19g2ldj2ks74jzlzsqq83640000gn/T/ipykernel_30298/249534669.py:77: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  pred = pred.fillna(method='ffill')


In [9]:
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
import jpholiday
import pandas as pd

# --- 1. 純粋な ARIMA モデルによる学習 -----------------------------
y = df['通常物数']

# p=5, d=1, q=2 の ARIMA
model = ARIMA(y, order=(5, 1, 2))
res = model.fit()
print(res.summary())

# --- 2. バックテスト（最後28日をホールドアウト） --------------------
# シンプルに get_prediction で直前実績を使わない固定ホライズン予測
hold = 28
train_end = len(y) - hold

res_bt = model.fit(start_params=res.params)  # 再フィット不要ですがサンプル
pred_bt = res_bt.get_prediction(start=train_end, end=len(y)-1)
pred_mean_bt = pred_bt.predicted_mean

actual = y.iloc[train_end:]
mae_bt  = mean_absolute_error(actual, pred_mean_bt)
rmse_bt = np.sqrt(mean_squared_error(actual, pred_mean_bt))
print(f"ARIMA Backtest MAE={mae_bt:.0f}, RMSE={rmse_bt:.0f}")

# --- 3. 将来28日予測 --------------------------------------------------
forecast = res.get_forecast(steps=365)
pred = forecast.predicted_mean

carry = 0.0
adjusted = pd.Series(index=pred.index, dtype=float)

# 2) ループで土日祝を繰り越し
for dt, raw_val in pred.items():
    # 祝日 or 土日判定
    is_wh = jpholiday.is_holiday(dt) or dt.weekday() >= 5

    if is_wh:
        # 繰り越しプールに溜めて、当日は 0
        carry += raw_val
        adjusted[dt] = 0.0
    else:
        # 平日は累積分を上乗せしてリセット
        adjusted[dt] = raw_val + carry
        carry = 0.0

# 3) 千通単位で四捨五入＆整数化
adjusted = adjusted.round(-3).astype(int)

# 4) 出力例
weekday_map = ['月','火','水','木','金','土','日']
for dt, v in adjusted.items():
    wd = weekday_map[dt.weekday()]
    mark = '祝' if jpholiday.is_holiday(dt) else ''
    print(f"{dt:%Y-%m-%d} ({wd}{mark}): {v} 通")


/Users/hiraokatatsuru/miniconda3/envs/shift_builder/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/hiraokatatsuru/miniconda3/envs/shift_builder/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/hiraokatatsuru/miniconda3/envs/shift_builder/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/hiraokatatsuru/miniconda3/envs/shift_builder/lib/python3.12/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregre

                               SARIMAX Results                                
Dep. Variable:                   通常物数   No. Observations:                 1339
Model:                 ARIMA(5, 1, 2)   Log Likelihood              -15350.286
Date:                Sat, 14 Jun 2025   AIC                          30716.572
Time:                        21:51:04   BIC                          30758.163
Sample:                    10-01-2021   HQIC                         30732.155
                         - 05-31-2025                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1         -0.0180      0.021     -0.855      0.392      -0.059       0.023
ar.L2         -0.6155      0.020    -31.360      0.000      -0.654      -0.577
ar.L3         -0.4337      0.023    -19.032      0.0

/Users/hiraokatatsuru/miniconda3/envs/shift_builder/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [10]:
import numpy as np
import pandas as pd
import jpholiday
from sklearn.metrics import mean_absolute_error, mean_squared_error
from statsmodels.tsa.arima.model import ARIMA

# --- 1. データ準備とモデル学習 ----------------------------------------
y = df['通常物数']

# 最終 28 日をホールドアウトしてバックテスト
hold = 28
train_end = len(y) - hold

# ARIMA(5,1,2) のフィッティング
model = ARIMA(y[:train_end], order=(5,1,2))
res = model.fit()

# --- 2. ホールドアウト期間の予測 --------------------------------------
# get_forecast で一括予測
forecast = res.get_forecast(steps=hold)
pred = forecast.predicted_mean
pred.index = y.index[train_end:]  # インデックスを合わせる

# --- 3. 土日祝キャリー適用関数 ----------------------------------------
def carry_over(series):
    carry = 0.0
    adjusted = pd.Series(index=series.index, dtype=float)
    for dt, raw in series.items():
        is_wh = jpholiday.is_holiday(dt) or dt.weekday() >= 5
        if is_wh:
            carry += raw
            adjusted[dt] = 0.0
        else:
            adjusted[dt] = raw + carry
            carry = 0.0
    return adjusted

# キャリー適用
pred_adj = carry_over(pred)

# --- 4. 指標計算 ------------------------------------------------------
actual = y.iloc[train_end:]
mae_before = mean_absolute_error(actual, pred)
rmse_before = np.sqrt(mean_squared_error(actual, pred))
mae_after  = mean_absolute_error(actual, pred_adj)
rmse_after = np.sqrt(mean_squared_error(actual, pred_adj))

print("バックテスト指標（キャリーなし）")
print(f"  MAE = {mae_before:.0f}, RMSE = {rmse_before:.0f}")
print("バックテスト指標（キャリーあり）")
print(f"  MAE = {mae_after:.0f}, RMSE = {rmse_after:.0f}")

# --- 5. キャリー後の予測表示 ------------------------------------------
pred_disp = pred_adj.round(-3).astype(int)
weekday_map = ['月','火','水','木','金','土','日']
for dt, v in pred_disp.items():
    wd = weekday_map[dt.weekday()]
    mark = '祝' if jpholiday.is_holiday(dt) else ''
    print(f"{dt:%Y-%m-%d} ({wd}{mark}): {v} 通")


/Users/hiraokatatsuru/miniconda3/envs/shift_builder/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/hiraokatatsuru/miniconda3/envs/shift_builder/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/hiraokatatsuru/miniconda3/envs/shift_builder/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/hiraokatatsuru/miniconda3/envs/shift_builder/lib/python3.12/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregre

バックテスト指標（キャリーなし）
  MAE = 21722, RMSE = 26089
バックテスト指標（キャリーあり）
  MAE = 8337, RMSE = 14006
2025-05-04 (日祝): 0 通
2025-05-05 (月祝): 0 通
2025-05-06 (火祝): 0 通
2025-05-07 (水): 155000 通
2025-05-08 (木): 42000 通
2025-05-09 (金): 18000 通
2025-05-10 (土): 0 通
2025-05-11 (日): 0 通
2025-05-12 (月): 80000 通
2025-05-13 (火): 42000 通
2025-05-14 (水): 45000 通
2025-05-15 (木): 35000 通
2025-05-16 (金): 22000 通
2025-05-17 (土): 0 通
2025-05-18 (日): 0 通
2025-05-19 (月): 78000 通
2025-05-20 (火): 44000 通
2025-05-21 (水): 44000 通
2025-05-22 (木): 35000 通
2025-05-23 (金): 22000 通
2025-05-24 (土): 0 通
2025-05-25 (日): 0 通
2025-05-26 (月): 79000 通
2025-05-27 (火): 44000 通
2025-05-28 (水): 44000 通
2025-05-29 (木): 34000 通
2025-05-30 (金): 22000 通
2025-05-31 (土): 0 通


In [13]:
import numpy as np
import pandas as pd
import jpholiday
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error, mean_squared_error

# --- 1. データ準備 --------------------------------------------
# y: 累積差分なしの時系列
y = df['通常物数']

# モデル定義に使う (p,d,q) は以前の ARIMA(5,1,2)、季節成分は週次 s=7 で (P,D,Q)
order            = (5, 1, 2)
seasonal_order   = (1, 0, 1, 7)  # 例：季節AR=1, 季節差分=0, 季節MA=1, 週次周期=7

# --- 2. SARIMA モデルの学習 ----------------------------------
model = SARIMAX(
    y,
    order=order,
    seasonal_order=seasonal_order,
    enforce_stationarity=False,
    enforce_invertibility=False
)
res = model.fit(disp=False)
print(res.summary())

# --- 3. バックテスト ------------------------------------------
# 最後28日をホールドアウトしてウォークフォワード予測
hold = 28
train_end = len(y) - hold

history = y.iloc[:train_end].copy()
pred_bt = []

for dt in y.index[train_end:]:
    # その時点までで再フィット（update() でも可、ここではシンプルに再フィット）
    m = SARIMAX(
        history,
        order=order,
        seasonal_order=seasonal_order,
        enforce_stationarity=False,
        enforce_invertibility=False
    ).fit(disp=False)
    
    # 1ステップ ahead forecast
    f = m.get_forecast(steps=1).predicted_mean.iloc[0]
    pred_bt.append(f)
    
    # 実績ではなく「予測」を history に追加すると動的再帰的テスト
    history.loc[dt] = f

actual = actual.values  # shape (hold,)
pred_array = np.array(pred_bt, dtype=float)

# 非 NaN の位置だけを使うマスクを作成
mask = ~np.isnan(pred_array)

# マスクを使ってフィルタリング
actual_filt    = actual[mask]
pred_filt      = pred_array[mask]

mae_bt  = mean_absolute_error(actual_filt, pred_filt)
rmse_bt = np.sqrt(mean_squared_error(actual_filt, pred_filt))

print(f"SARIMA Backtest MAE={mae_bt:.0f}, RMSE={rmse_bt:.0f}")

# --- 4. 将来28日予測＋土日祝キャリー --------------------------
# 本番学習は全データで一度だけ
dall_model = SARIMAX(
    y,
    order=order,
    seasonal_order=seasonal_order,
    enforce_stationarity=False,
    enforce_invertibility=False
).fit(disp=False)

future_dates = pd.date_range(start=df.index[-1] + pd.Timedelta(days=1), periods=28, freq='D')
pred = dall_model.get_forecast(steps=365).predicted_mean
pred.index = future_dates

# carry-over 関数
def carry_over(series):
    carry = 0.0
    adjusted = pd.Series(index=series.index, dtype=float)
    for dt, raw in series.items():
        is_wh = jpholiday.is_holiday(dt) or dt.weekday() >= 5
        if is_wh:
            carry += raw
            adjusted[dt] = 0.0
        else:
            adjusted[dt] = raw + carry
            carry = 0.0
    return adjusted

pred_adj = carry_over(pred).round(-3).astype(int)

# 表示
weekday_map = ['月','火','水','木','金','土','日']
for dt, v in pred_adj.items():
    mark = '祝' if jpholiday.is_holiday(dt) else ''
    print(f"{dt:%Y-%m-%d} ({weekday_map[dt.weekday()]}{mark}): {v} 通")


/Users/hiraokatatsuru/miniconda3/envs/shift_builder/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/hiraokatatsuru/miniconda3/envs/shift_builder/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/hiraokatatsuru/miniconda3/envs/shift_builder/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                                      SARIMAX Results                                      
Dep. Variable:                                通常物数   No. Observations:                 1339
Model:             SARIMAX(5, 1, 2)x(1, 0, [1], 7)   Log Likelihood              -15242.044
Date:                             Sat, 14 Jun 2025   AIC                          30504.088
Time:                                     22:06:43   BIC                          30555.987
Sample:                                 10-01-2021   HQIC                         30523.542
                                      - 05-31-2025                                         
Covariance Type:                               opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1         -0.9652      0.043    -22.358      0.000      -1.050      -0.881
ar.L2         -0.1249      

/Users/hiraokatatsuru/miniconda3/envs/shift_builder/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/hiraokatatsuru/miniconda3/envs/shift_builder/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/hiraokatatsuru/miniconda3/envs/shift_builder/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/hiraokatatsuru/miniconda3/envs/shift_builder/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/h

AttributeError: 'numpy.ndarray' object has no attribute 'values'

In [16]:
# 1) 365日先の日付インデックスを作成
start_date   = df.index[-1] + pd.Timedelta(days=1)
future_dates = pd.date_range(start=start_date, periods=365, freq='D')

# 2) まとめて予測
forecast = res.get_forecast(steps=365)
pred     = forecast.predicted_mean
pred.index = future_dates  # インデックスを合わせる

# 3) 土日祝キャリー関数（再掲）
def carry_over(series):
    carry    = 0.0
    adjusted = pd.Series(index=series.index, dtype=float)
    for dt, raw in series.items():
        is_wh = jpholiday.is_holiday(dt) or dt.weekday() >= 5
        if is_wh:
            carry += raw
            adjusted[dt] = 0.0
        else:
            adjusted[dt] = raw + carry
            carry = 0.0
    return adjusted

# 4) キャリー適用 + 千通単位四捨五入 + 整数化
pred_adj = carry_over(pred).round(-3).astype(int)



# もし全件を一覧で見たい場合は
for dt, v in pred_adj.items():
    wd = ['月','火','水','木','金','土','日'][dt.weekday()]
    mark = '祝' if jpholiday.is_holiday(dt) else ''
    print(f"{dt:%Y-%m-%d} ({wd}{mark}): {v} 通")

2025-06-01 (日): 0 通
2025-06-02 (月): 70000 通
2025-06-03 (火): 24000 通
2025-06-04 (水): 41000 通
2025-06-05 (木): 35000 通
2025-06-06 (金): 38000 通
2025-06-07 (土): 0 通
2025-06-08 (日): 0 通
2025-06-09 (月): 70000 通
2025-06-10 (火): 25000 通
2025-06-11 (水): 41000 通
2025-06-12 (木): 35000 通
2025-06-13 (金): 38000 通
2025-06-14 (土): 0 通
2025-06-15 (日): 0 通
2025-06-16 (月): 70000 通
2025-06-17 (火): 25000 通
2025-06-18 (水): 41000 通
2025-06-19 (木): 34000 通
2025-06-20 (金): 38000 通
2025-06-21 (土): 0 通
2025-06-22 (日): 0 通
2025-06-23 (月): 70000 通
2025-06-24 (火): 25000 通
2025-06-25 (水): 41000 通
2025-06-26 (木): 34000 通
2025-06-27 (金): 38000 通
2025-06-28 (土): 0 通
2025-06-29 (日): 0 通
2025-06-30 (月): 70000 通
2025-07-01 (火): 25000 通
2025-07-02 (水): 40000 通
2025-07-03 (木): 34000 通
2025-07-04 (金): 37000 通
2025-07-05 (土): 0 通
2025-07-06 (日): 0 通
2025-07-07 (月): 70000 通
2025-07-08 (火): 24000 通
2025-07-09 (水): 40000 通
2025-07-10 (木): 34000 通
2025-07-11 (金): 37000 通
2025-07-12 (土): 0 通
2025-07-13 (日): 0 通
2025-07-14 (月): 7000

In [17]:
def backtest_sarima_dynamic(df, order, seasonal_order, horizon=28):
    """
    df:               DataFrame(index=日時, '通常物数' 列あり)
    order:            (p,d,q) のタプル
    seasonal_order:   (P,D,Q,s) のタプル
    horizon:          バックテスト日数
    """
    y = df['通常物数']
    train_end = len(y) - horizon
    history   = y.iloc[:train_end].copy()
    carry     = 0.0
    preds     = []

    for dt in y.index[train_end:]:
        # 1. その時点までで SARIMA を再フィット
        model = SARIMAX(
            history,
            order=order,
            seasonal_order=seasonal_order,
            enforce_stationarity=False,
            enforce_invertibility=False
        ).fit(disp=False)

        # 2. 1ステップ予測
        raw = model.get_forecast(steps=1).predicted_mean.iloc[0]

        # 3. 土日祝キャリー処理
        is_wh = jpholiday.is_holiday(dt) or dt.weekday() >= 5
        if is_wh:
            carry += raw
            pred = 0.0
        else:
            pred = raw + carry
            carry = 0.0

        preds.append(pred)
        history.loc[dt] = pred  # ラグ用に追加

    # --- 指標計算 ---
    actual = y.iloc[train_end:]
    pred_arr = np.array(preds, dtype=float)
    mae  = mean_absolute_error(actual, pred_arr)
    rmse = np.sqrt(mean_squared_error(actual, pred_arr))

    return mae, rmse, pd.Series(preds, index=y.index[train_end:])

# ---------------------------------------
# 実行例
# ---------------------------------------
order          = (5, 1, 2)
seasonal_order = (1, 0, 1, 7)  # 週次サイクル

mae_sarima, rmse_sarima, sarima_preds = backtest_sarima_dynamic(
    df, order, seasonal_order, horizon=28
)
print(f"SARIMA Dynamic Backtest — MAE: {mae_sarima:.0f}, RMSE: {rmse_sarima:.0f}")

/Users/hiraokatatsuru/miniconda3/envs/shift_builder/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/hiraokatatsuru/miniconda3/envs/shift_builder/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/hiraokatatsuru/miniconda3/envs/shift_builder/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/hiraokatatsuru/miniconda3/envs/shift_builder/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/h

SARIMA Dynamic Backtest — MAE: 894571, RMSE: 2241409


/Users/hiraokatatsuru/miniconda3/envs/shift_builder/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
